# WPP: total population

## Parameters

In [ ]:
dest_dir = "/tmp/wpp_2019_total_population"

## Walden

In [ ]:
from owid import walden

In [ ]:
walden_ds = walden.Catalog().find_one("wpp", "2019", "standard_projections")

In [ ]:
walden_ds

## Unzip

In [ ]:
import tempfile
import shutil

In [ ]:
temp_dir = tempfile.mkdtemp()

In [ ]:
import zipfile

In [ ]:
zipfile.ZipFile(walden_ds.local_path).extractall(temp_dir)

In [ ]:
!ls {temp_dir}/WPP2019

## Make dataset

In [ ]:
from owid.catalog import Dataset
from etl.steps.data import converters

In [ ]:
ds = Dataset.create_empty(dest_dir)

In [ ]:
ds.metadata = converters.convert_walden_metadata(walden_ds)

In [ ]:
ds.save()

## Add tables

In [ ]:
from owid.catalog import Table
import pandas as pd

### Total population

In [ ]:
df = pd.read_csv(f"{temp_dir}/WPP2019/WPP2019_TotalPopulationBySex.csv")

In [ ]:
df.head()

In [ ]:
df.columns = [
    "loc_id",
    "location",
    "var_id",
    "variant",
    "year",
    "mid_period",
    "population_male",
    "population_female",
    "population_total",
    "population_density",
]

In [ ]:
t = Table(df[["loc_id", "location"]].drop_duplicates().set_index("loc_id"))
t.metadata.short_name = "location_codes"
ds.add(t)

In [ ]:
t = Table(df[["var_id", "variant"]].drop_duplicates().set_index("var_id"))
t.metadata.short_name = "variant_codes"
ds.add(t)

In [ ]:
df.drop(columns=["loc_id", "var_id"], inplace=True)

In [ ]:
for col in ["location", "variant"]:
    df[col] = df[col].astype("category")

In [ ]:
df.set_index(["variant", "location", "year"], inplace=True)

In [ ]:
df

In [ ]:
df.index.levels[0]

In [ ]:
t = Table(df)
t.metadata.short_name = "total_population"
ds.add(t)

### Fertility by age

In [ ]:
df = pd.read_csv(f"{temp_dir}/WPP2019/WPP2019_Fertility_by_Age.csv")

In [ ]:
df.head()

In [ ]:
df.drop(
    columns=["LocID", "VarID", "MidPeriod", "AgeGrpStart", "AgeGrpSpan"], inplace=True
)

In [ ]:
df.columns = [
    "location",
    "variant",
    "year_range",
    "age_group",
    "asfr",
    "pasfr",
    "births",
]

In [ ]:
df.head()

In [ ]:
for col in ["location", "variant", "year_range", "age_group"]:
    df[col] = df[col].astype("category")

In [ ]:
df.set_index(["variant", "location", "year_range", "age_group"], inplace=True)

In [ ]:
t = Table(df)
t.metadata.short_name = "fertility_by_age"
ds.add(t)

### Population by age and sex

In [ ]:
df = pd.read_csv(f"{temp_dir}/WPP2019/WPP2019_PopulationByAgeSex_Medium.csv")

In [ ]:
df.head()

In [ ]:
df.drop(
    columns=["LocID", "VarID", "MidPeriod", "AgeGrpStart", "AgeGrpSpan"], inplace=True
)

In [ ]:
df.columns = [
    "location",
    "variant",
    "year",
    "age_group",
    "population_male",
    "population_female",
    "population_total",
]

In [ ]:
df.head()

In [ ]:
for col in ["location", "variant", "age_group"]:
    df[col] = df[col].astype("category")

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df.set_index(["variant", "location", "year", "age_group"], inplace=True)

In [ ]:
df.head()

In [ ]:
t = Table(df)
t.metadata.short_name = "population_by_age_sex"
ds.add(t)

## Clean up

In [ ]:
shutil.rmtree(temp_dir)